# Intialisation Steps
Connecting to Snowflake

## Importing required modules

In [22]:
import pandas as pd
from drg_connect import Snowflake
import qgrid 
from datetime import timedelta, datetime
import math

import warnings
warnings.filterwarnings('ignore')

## Snowflake connection parameter

In [23]:
##defining parameters of snowflake
snow = Snowflake(role = 'RWD_ANALYTICS_RW',database='SANDBOX_ANALYTICS',schema = 'SANDBOX')
engine = snow.engine

%reload_ext sql_magic
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine

# Request details

They are looking for number of patients with Enzyme Replace Therapy in MPS I population

## Ref table for MPS I population

In [3]:
snow.select("select * from RWD_DB.RWD.ICD_GROUPER limit 3")

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


,id,level_1,level_1_description,level_2,level_2_description,level_3,level_3_description,level_4,icd9_mapped_codes,icd9_description,level_4_short_description_icd10,level_4_long_description_icd10,create_ts,update_ts
0,1,A00 - B999,Certain infectious and parasitic diseases,A00-A09,Intestinal infectious diseases,A00-A009,Cholera,A00,No map,No map,Cholera,Cholera,2017-11-02,2017-11-02
1,2,A00 - B999,Certain infectious and parasitic diseases,A00-A09,Intestinal infectious diseases,A00-A009,Cholera,A000,0010,Cholera due to vibrio cholerae,Cholera due to Vibrio cholerae 01 biovar cholerae,Cholera due to Vibrio cholerae 01 biovar cholerae,2017-11-02,2017-11-02
2,3,A00 - B999,Certain infectious and parasitic diseases,A00-A09,Intestinal infectious diseases,A00-A009,Cholera,A001,0011,Cholera due to vibrio cholerae el tor,Cholera due to Vibrio cholerae 01 biovar eltor,Cholera due to Vibrio cholerae 01 biovar eltor,2017-11-02,2017-11-02


In [4]:
%%read_sql

select
    *
from
    RWD_DB.RWD.ICD_GROUPER
where
    level_4_short_description_icd10 ilike '%mucopolysaccharidosis%'

Query started at 02:20:24 PM India Standard Time; Query executed in 0.08 m

,id,level_1,level_1_description,level_2,level_2_description,level_3,level_3_description,level_4,icd9_mapped_codes,icd9_description,level_4_short_description_icd10,level_4_long_description_icd10,create_ts,update_ts
0,4667,E00 - E8989,Endocrine nutritional and metabolic diseases,E70-E88,Metabolic disorders,E76-E769,Disorders of glycosaminoglycan metabolism,E760,No map,No map,Mucopolysaccharidosis type I,Mucopolysaccharidosis type I,2017-11-02,2017-11-02
1,4671,E00 - E8989,Endocrine nutritional and metabolic diseases,E70-E88,Metabolic disorders,E76-E769,Disorders of glycosaminoglycan metabolism,E761,2775,Mucopolysaccharidosis,Mucopolysaccharidosis type II,Mucopolysaccharidosis type II,2017-11-02,2017-11-02
2,4679,E00 - E8989,Endocrine nutritional and metabolic diseases,E70-E88,Metabolic disorders,E76-E769,Disorders of glycosaminoglycan metabolism,E763,2775,Mucopolysaccharidosis,Mucopolysaccharidosis unspecified,Mucopolysaccharidosis unspecified,2017-11-02,2017-11-02


## Ref table for enzyme replacement therapy

In [9]:
%%read_sql

select
    *
from
    RWD_DB.RWD_REFERENCE_LIBRARY.HCPCS_BASE_LIBRARY
where 
     (full_description ilike '%enzyme%'
      and full_description ilike '%replace%')
      --and long_description ilike '%therapy%')

Query started at 02:31:40 PM India Standard Time; Query executed in 0.09 m

,hcpcs_code,short_description,long_description,full_description,status,source_file_name
0,S9357,HIT ENZYME REPL IV TX; PER DIEM,HOME INFUSION TX ENZYME REPL IV TX; PER DIEM,"Home infusion therapy, enzyme replacement intr...",None,OPTUM_HCPCS_BASE_2014_07.zip
1,S9357,HIT ENZYME REPL IV TX; PER DIEM,HOME INFUSION TX ENZYME REPL IV TX; PER DIEM,"Home infusion therapy, enzyme replacement intr...",None,OPTUM_HCPCS_BASE_2014_10.zip
2,S9357,HIT ENZYME REPL IV TX; PER DIEM,HOME INFUSION TX ENZYME REPL IV TX; PER DIEM,"Home infusion therapy, enzyme replacement intr...",None,OPTUM_HCPCS_BASE_2014_07_revised.zip
3,S9357,HIT ENZYME REPL IV TX; PER DIEM,HOME INFUSION TX ENZYME REPL IV TX; PER DIEM,"Home infusion therapy, enzyme replacement intr...",None,OPTUM_HCPCS_BASE_2015_01.zip
4,S9357,HIT ENZYME REPL IV TX; PER DIEM,HOME INFUSION TX ENZYME REPL IV TX; PER DIEM,"Home infusion therapy, enzyme replacement intr...",None,OPTUM_HCPCS_BASE_2015_04_update.zip
5,S9357,HIT ENZYME REPL IV TX; PER DIEM,HOME INFUSION TX ENZYME REPL IV TX; PER DIEM,"Home infusion therapy, enzyme replacement intr...",None,OPTUM_HCPCS_BASE_2015_07.zip
6,S9357,HIT ENZYME REPL IV TX; PER DIEM,HOME INFUSION TX ENZYME REPL IV TX; PER DIEM,"Home infusion therapy, enzyme replacement intr...",None,OPTUM_HCPCS_BASE_2016_01_Update.zip
7,S9357,HIT ENZYME REPL IV TX; PER DIEM,HOME INFUSION TX ENZYME REPL IV TX; PER DIEM,"Home infusion therapy, enzyme replacement intr...",None,OPTUM_HCPCS_BASE_2015_10.zip
8,S9357,HIT ENZYME REPL IV TX; PER DIEM,HOME INFUSION TX ENZYME REPL IV TX; PER DIEM,"Home infusion therapy, enzyme replacement intr...",None,OPTUM_HCPCS_BASE_2016_04.zip
9,S9357,HIT ENZYME REPL IV TX; PER DIEM,HOME INFUSION TX ENZYME REPL IV TX; PER DIEM,"Home infusion therapy, enzyme replacement intr...",None,OPTUM_HCPCS_BASE_2016_07.zip


# Getting cohort and proportions

## Selecting patients with MPS I

In [34]:
%%read_sql
create or replace table st_mps1 as
select
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
    left(year_of_service, 4) as year,
    claim_number,
    diagnosis,
    diagnosis_sequence
    
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_DIAGNOSIS
where
    diagnosis in ('E760', 'E7601', 'E7602', 'E7603')

Query started at 07:22:03 PM India Standard Time; Query executed in 10.03 m

,status
0,Table ST_MPS1 successfully created.


In [45]:
%%read_sql
select count (distinct patient_id)
from
    st_mps1
    
where 
    left(patient_id, 5) != 'XXX -'

Query started at 08:27:16 PM India Standard Time; Query executed in 0.11 m

,COUNT (DISTINCT PATIENT_ID)
0,1383


In [35]:
%%read_sql
select year,
       count (distinct patient_id)
from
    st_mps1
    
where 
    left(patient_id, 5) != 'XXX -'
group by 1
order by 1

Query started at 07:32:11 PM India Standard Time; Query executed in 0.07 m

,year,COUNT (DISTINCT PATIENT_ID)
0,2013,1
1,2015,265
2,2016,675
3,2017,656
4,2018,462


## Selecting enzyme replacement therapy HCPCS codes

In [13]:
snow.select ("select * from RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PROCEDURE limit 30")

,claim_number,encrypted_key_1,encrypted_key_2,procedure_type,procedure,procedure_sequence,line_number,statement_from,statement_to,service_from,...,upc_cde,fingertip_parent_id,fingertip_parent_name,fingertip_payer_id,fingertip_payer_name,data_source,load_indicator,drg_patient_id,etl_create_ts,etl_update_ts
0,298625000000344803,xdPj2Szhm2nu8CS6JXwq17/PGpmWxAXmLjDtQ8LdN7A=,mtHMMAnNvdph/e5HJ+PT6+yk67jLSInAjWILPDRXg54=,CPT,84703,5,5,2018-07-12,2018-07-12,2018-07-12,...,None,None,None,None,None,OSPREY,I,None,2018-08-24,2018-08-24
1,298625000000152693,E2OV7LDKnjhCkuOHWq35HC20QAV6pzpRrvqGa/0mh8s=,fmlo309Ri6QiwG8MlbA9B6yl5VZ1CUIR8DlmGhNYeHs=,CPT,J2704,23,23,2018-08-01,2018-08-02,2018-08-01,...,None,None,None,None,None,OSPREY,I,None,2018-08-24,2018-08-24
2,298625000000342010,Gwj139bdVV5483ATKSWvXV4fFeXtDExNylVpriEoEWk=,sf8rQTImc0PgX/gLXoZCnOgR8IwV72BxGVjk8IzESEQ=,CPT,None,18,18,2017-02-21,2017-03-03,None,...,None,None,None,None,None,OSPREY,I,None,2018-08-24,2018-08-24
3,298625000000237728,HIe9lnK0JwCLSeJPek8KIfHoi8jORhpOOBkR1N2PN14=,4My+z98VQjYKJnCZAdY2hJu13n9290if1hd8q+a/D7Y=,CPT,85018,2,2,2018-08-16,2018-08-16,2018-08-16,...,None,None,None,None,None,OSPREY,I,None,2018-08-24,2018-08-24
4,298624000000096921,fqYuu3WXqTVUy8keNg5Ntc53gQiBfwbeTh9gv10lIFU=,gKckPQzgaaOYqS/v5BZCflkU/yCzBaOxUGz4GRqHUxo=,CPT,93010,1,1,None,None,2018-08-06,...,None,None,None,None,None,OSPREY,I,None,2018-08-24,2018-08-24
5,298624000000177135,ZKLYvHAwUoXJt5i4Dpbf+GS/dN0AFdP7S4iO/we1zOU=,0ya3D60llOLWkGV5zgAT2TSAXVAPGtlHYMxjCltkmCM=,CPT,99212,1,1,None,None,2018-08-20,...,None,None,None,None,None,OSPREY,I,None,2018-08-24,2018-08-24
6,298625000000362146,9HfEGxicmHDO3ryB2D5AgWdnA3npcS7TIxT8LOsMZj8=,qVRl9LORZnCyja/IjnQMT7rfD2TAREaH+bDdGbet+9w=,CPT,80076,4,4,2018-06-11,2018-06-11,2018-06-11,...,None,None,None,None,None,OSPREY,I,None,2018-08-24,2018-08-24
7,298625000000077431,X/tl4NoFALCu827L6fJwCltlGACu47qRCDeUd6hmKCw=,EO7OPJGK93IFPihQZI+R3DYNFJRWJoMPN4JAgudR/Ds=,CPT,85027,2,2,2018-06-21,2018-06-21,2018-06-21,...,None,None,None,None,None,OSPREY,I,None,2018-08-24,2018-08-24
8,298625000000436644,rFbAfJTakYY3HwvNLIei5t7SOghNfZIQEr4dhIe+tz0=,pJgUeqpNdpD+RA8z+9OgPSTEO7dtb98gC5mH7gRRylw=,CPT,G0299,2,2,2018-08-01,2018-08-08,2018-08-02,...,None,None,None,None,None,OSPREY,I,None,2018-08-24,2018-08-24
9,298625000000213701,YbmgSNL2unYWOtbgKpQN71OqK+d8R70Rqctksbi3J1g=,u2PtCJoE1thGUfDQmJ6KL4jUkLOLh2zpipbkzj62Vj0=,CPT,None,1,1,2018-07-01,2018-07-31,2018-07-01,...,None,None,None,None,None,OSPREY,I,None,2018-08-24,2018-08-24


In [39]:
%%read_sql
create or replace table st_mps1_cpt as
select
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
    procedure
    
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PROCEDURE
where
    --procedure_type ilike '%CPT%'
    procedure in ('S9357', 'J1931')

Query started at 08:03:46 PM India Standard Time; Query executed in 10.69 m

,status
0,Table ST_MPS1_CPT successfully created.


In [40]:
%%read_sql
select procedure,
       count (distinct patient_id)
from
    st_mps1_cpt
    
where 
    left(patient_id, 5) != 'XXX -'
group by 1
order by 1

Query started at 08:18:02 PM India Standard Time; Query executed in 0.13 m

,procedure,COUNT (DISTINCT PATIENT_ID)
0,J1931,167
1,S9357,1138


# Joining tables to get proportions

In [42]:
%%read_sql

select
    year,
     count (distinct patient_id)
 from
    st_mps1
where
     patient_id in (select  procedure, (distinct patient_id) 
                      from st_mps1_cpt)                    
group by 1
order by 1

Query started at 08:19:17 PM India Standard Time

ProgrammingError: (snowflake.connector.errors.ProgrammingError) 001003 (42000): SQL compilation error:
syntax error line 7 at position 20 unexpected 'select'.
syntax error line 7 at position 40 unexpected 'distinct'.
syntax error line 8 at position 22 unexpected 'from'. [SQL: 'select\n    year,\n     count (distinct patient_id)\n from\n    st_mps1\nwhere\n     patient_id in (select  procedure, (distinct patient_id) \n                      from st_mps1_cpt)                    \ngroup by 1\norder by 1'] (Background on this error at: http://sqlalche.me/e/f405)

In [44]:
%%read_sql
select procedure,
       count (distinct patient_id)
from
    st_mps1_cpt
    
where 
     patient_id in (select distinct patient_id from st_mps1)
     
    group by 1
    order by 1

Query started at 08:21:50 PM India Standard Time; Query executed in 0.08 m

,procedure,COUNT (DISTINCT PATIENT_ID)
0,J1931,129
1,S9357,53


# Checking other related codes

In [28]:
%%read_sql
create or replace table st_mps as
select
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
    left(year_of_service, 4) as year,
    claim_number,
    diagnosis,
    diagnosis_sequence
    
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_DIAGNOSIS
where
    diagnosis in ('E760', 'E761', 'E763', '2775')

Query started at 04:49:12 PM India Standard Time; Query executed in 11.50 m

,status
0,Table ST_MPS successfully created.


In [31]:
%%read_sql
select diagnosis,
       year,
       count (distinct patient_id)
from
    st_mps
    
where 
    left(patient_id, 5) != 'XXX -'
    
group by diagnosis, year
order by diagnosis, year

Query started at 05:02:03 PM India Standard Time; Query executed in 0.08 m

,diagnosis,year,COUNT (DISTINCT PATIENT_ID)
0,2775,2001,3
1,2775,2007,1
2,2775,2009,4
3,2775,2010,12
4,2775,2011,198
5,2775,2012,913
6,2775,2013,1892
7,2775,2014,1916
8,2775,2015,1811
9,2775,2016,1


In [38]:
%%read_sql

select
    *
from
RWD_DB.RWD_REFERENCE_LIBRARY.IG_PHARMACY_CODE_BOOK limit 5

Query started at 07:55:19 PM India Standard Time; Query executed in 0.11 m

,code_type,code,description,field_number_possibly_in_error,field_limitation_or_explanation
0,Reject,00,M/I Means Missing/Invalid,None,None
1,Reject,01,M/I Bin Number,101-A1,None
2,Reject,02,M/I Version/Release Number,102-A2,None
3,Reject,03,M/I Transaction Code,103-A3,None
4,Reject,04,M/I Processor Control Number,104-A4,None
